* GPu 사용량 14.1G/15G
* CPU 사용량 3.9G/12.7G
*   1.2 hr (1 epoch)



In [1]:
from google.colab import drive
drive.mount("/gdrive",force_remount=True)

Mounted at /gdrive


In [2]:
!pip install accelerate -U
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00


In [1]:
!pip install transformers dataset evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
 

In [2]:
! pip install huggingface_hub

In [3]:
from huggingface_hub import notebook_login

notebook_login()

원본 쥬피터 노트북 주소
[translation_mt5](http://ethen8181.github.io/machine-learning/deep_learning/seq2seq/translation_mt5/translation_mt5.html)

In [4]:
from datasets import load_dataset
raw_ds = load_dataset("bongsoo/news_talk_en_ko",split="train").select(range(250000,450000))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
raw_ds

Dataset({
    features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
    num_rows: 200000
})

In [6]:
split_datasets = raw_ds.train_test_split(train_size=0.99,seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 198000
    })
    test: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 2000
    })
})

In [7]:
split_datasets["validation"] = split_datasets.pop("test")

In [8]:
split_datasets = split_datasets.rename_column("Skinner's reward is mostly eye-watering.",'en')
split_datasets = split_datasets.rename_column("스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.",'ko')

In [9]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 198000
    })
    validation: Dataset({
        features: ['en', 'ko'],
        num_rows: 2000
    })
})

In [10]:
sample = split_datasets["train"][2]
sample

{'en': 'The reason was that the taste of the fruit given by my mother-in-law was not good.',
 'ko': '이유는 장모가 대주는 과일 맛이 별로였던 것이다.'}

In [11]:
!pip install rouge_score
!pip install sacrebleu

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=78e10e3e469c07cba2588872fd16ff07a71e2c788f20ad171db13b8971dab520
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.3 MB/s eta 0:00:00


In [12]:
import evaluate
rouge_score = evaluate.load("rouge")
bleu_score = evaluate.load("bleu")
sacrebleu_score = evaluate.load("sacrebleu")

In [13]:
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"

In [14]:
# as we can see the metric returns necessary components including
# 'bleu': bleu score
# 'precisions': geometric mean of n-gram precisions
# 'brevity_penalty': brevity penalty
# we can confirm precision 1 is indeed 6 / 7
bleu_score.compute(
    predictions=[generated_summary],
    references=[reference_summary]
)

{'bleu': 0.6434588841607617,
 'precisions': [0.8571428571428571, 0.6666666666666666, 0.6, 0.5],
 'brevity_penalty': 1.0,
 'length_ratio': 1.1666666666666667,
 'translation_length': 7,
 'reference_length': 6}

In [15]:
# result from another bleu implementation
sacrebleu_score.compute(
    predictions=[generated_summary],
    references=[reference_summary]
)

{'score': 64.34588841607616,
 'counts': [6, 4, 3, 2],
 'totals': [7, 6, 5, 4],
 'precisions': [85.71428571428571, 66.66666666666667, 60.0, 50.0],
 'bp': 1.0,
 'sys_len': 7,
 'ref_len': 6}

In [16]:
# rouge 1:
# precision = 6 / 7
# recall = 6 / 6
# 2 * (precision * recall) / (recall + precision)
scores = rouge_score.compute(
    predictions=[generated_summary],
    references=[reference_summary],
    rouge_types=["rouge1", "rouge2", "rougeL"]
)
scores

{'rouge1': 0.923076923076923,
 'rouge2': 0.7272727272727272,
 'rougeL': 0.923076923076923}

In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_ckpt="chunwoolee0/mt5_small_bongsoo_en_ko"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

print("number of parameters:", model.num_parameters())

number of parameters: 300176768


In [18]:
import numpy as np
import torch
import multiprocessing

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [20]:
NUM_CPU = multiprocessing.cpu_count()
NUM_CPU

2

In [21]:
max_length = 64
def batch_tokenize_fn(examples):
    """
    Generate the input_ids and labels field for huggingface dataset/dataset dict.

    Truncation is enabled where we cap the sentence to the max length. Padding will be done later
    in a data collator, so we pad examples to the longest length within a mini-batch and not
    the whole dataset.
    """
    sources = examples["en"]
    targets = examples["ko"]
    model_inputs = tokenizer(sources, max_length=max_length, truncation=True)

    # setup the tokenizer for targets,
    # huggingface expects the target tokenized ids to be stored in the labels field
    # note, newer version of tokenizer supports a text_target argument, where we can create
    # source and target sentences in one go
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [22]:
tokenized_datasets = split_datasets.map(
    batch_tokenize_fn,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
    num_proc=NUM_CPU
)
tokenized_datasets

Map (num_proc=2):   0%|          | 0/198000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 198000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [23]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

features = [tokenized_datasets["train"][i] for i in range(2)]
output = data_collator(features)
output

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[   259, 124843,    339,  16857,    347,    351,    287,    317, 146206,
          34966,    263,    304,    412,  26256,   1525,    264, 159566,    261,
           1866,    259,  24083,    351,   1001,   2262,  28018,  35189,   3026,
            313,  23192, 122173,    347,   6478,   2454,      1,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0],
        [ 13976,  29716,   2426,    281,  13203,   2173,    261,    313,   1675,
            259,   2220,   6557,    288,   3534,    259,  17250,  17114,    288,
            259,  81047,    287,  71710,  29825,   7176,    754,   1156,   5022,
           4801,    305,    287,  13749,   2426,    533,   2108,    339,    375,
          60557,  30353,   2127,    288,   5193,    287,  71710,  29825,   3004,
           2387,  25236,    344,    287,   3265,    260,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 

In [24]:
from transformers import Seq2SeqTrainingArguments

fine_tuned_model_checkpoint = "mt5_small_bongsoo_en_ko"
output_dir = fine_tuned_model_checkpoint
batch_size = 32

args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="steps",
    learning_rate=2.e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    load_best_model_at_end=True,
    greater_is_better=True,
    metric_for_best_model="rougeL",
    gradient_accumulation_steps=2,
    # careful when attempting to train t5 models on fp16 mixed precision,
    # the model was trained on bfloat16 mixed precision, and mixing different mixed precision
    # type might result in nan loss
    # https://discuss.huggingface.co/t/mixed-precision-for-bfloat16-pretrained-models/5315
    fp16=False,
    push_to_hub=True,
)

In [25]:
import numpy as np
def compute_metrics(eval_pred):
    """
    Compute rouge and bleu metrics for seq2seq model generated prediction.

    tip: we can run trainer.predict on our eval/test dataset to see what a sample
    eval_pred object would look like when implementing custom compute metrics function
    """
    predictions, labels = eval_pred
    # Decode generated summaries, which is in ids into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode labels, a.k.a. reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_score.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        rouge_types=["rouge1", "rouge2", "rougeL"]
    )
    score = sacrebleu_score.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )
    result["sacrebleu"] = score["score"]
    return {k: round(v, 4) for k, v in result.items()}

In [26]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [27]:
# should take around 4117.78 seconds on a single V100 GPU
from time import perf_counter
t1_start = perf_counter()
train_output = trainer.train()
t1_stop = perf_counter()
print("Training elapsed time:", t1_stop - t1_start)

# saving the model which allows us to leverage
# .from_pretrained(model_path)

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Sacrebleu
500,3.833800,2.962588,0.147500,0.018400,0.145500,0.424300
1000,3.786500,2.930534,0.152900,0.018100,0.150800,0.443500
1500,3.743600,2.906702,0.157200,0.019000,0.155000,0.446400
2000,3.720700,2.892374,0.165000,0.023300,0.162900,0.453200
2500,3.702200,2.882455,0.164700,0.023100,0.162700,0.450400
3000,3.690000,2.877817,0.166200,0.023700,0.164700,0.469400


Training elapsed time: 5286.103062483


In [28]:
trainer.evaluate()

{'eval_loss': 2.877816915512085,
 'eval_rouge1': 0.1662,
 'eval_rouge2': 0.0237,
 'eval_rougeL': 0.1647,
 'eval_sacrebleu': 0.4694,
 'eval_runtime': 36.6209,
 'eval_samples_per_second': 54.614,
 'eval_steps_per_second': 1.72,
 'epoch': 1.0}

In [29]:
trainer.push_to_hub()

'https://huggingface.co/chunwoolee0/mt5_small_bongsoo_en_ko/tree/main/'

In [30]:
from transformers import pipeline
model_ckpt = "chunwoolee0/mt5_small_bongsoo_en_ko"
translator = pipeline("translation", model=model_ckpt)

In [31]:
translator("Skinner's reward is mostly eye-watering.")

[{'translation_text': '벤더의 선물은 너무 마음이 쏠린다.'}]

In [32]:
translator("I go for a walk after lunch.")

[{'translation_text': '식당에 앉아서 밤에 갔다.'}]

In [33]:
translator("This is a fairly technical error in which a required device for network communications (a socket) could not be created.", max_length=400)

[{'translation_text': '통신통신(이메라)가 적용되지 않은 상황이 사실상 사실상 사실상 사실이다.'}]

In [34]:
translator("To him, literature is a companion that inspires the way of inspiration whenever the idea is blocked, in last year, he appeared as an actor in the movie 'Emperor', which was invited to the Pusan International Film Festival.", max_length=400)

[{'translation_text': '지난해 한 일간 미술은 이미지가 삭제되면서 미술은 이미지가 삭제되면서 미술이 삭제되면서 배우가 배우로 배우로 배우로 배우로 배우로 배우로 배우로 배우로 배우로 배우됐다.'}]